In [1]:
#Tensorflow
import tensorflow as tf
from tensorflow.keras import Sequential
from tensorflow import keras
from tensorflow.keras.models import Model
from tensorflow.keras.models import load_model
from tensorflow.keras import layers, activations
from tensorflow.keras.layers import *
from tensorflow.keras.optimizers import *

#Pandas and numpy for data formats
import pandas as pd
import numpy as np


#glob for data import
import glob
import random
#PyTorch
import torch
import torch.nn as nn
#SK Learn
import sklearn as skl
from sklearn.model_selection import train_test_split
from sklearn import preprocessing
from sklearn.preprocessing import StandardScaler
#from sklearn.utils import shuffle
import math
import sys
#MatPlotLib
import matplotlib.pyplot as plt

from models import simpleRegression
from models import feedForward, sequentialModel, sequentialDropout, lstmModel, cnnModel, cnnModelFilters, scoringSystem

#import functions
from dataImport import importTrainingData, importParamsData, importTestData, importTrainParameter, importTargetParameter, importEvalParameter
from preprocessing import preprocessData, createTrainTensors, createParamsTensors, preprocessParams, rescaleData, preprocessParameters

INFO:tensorflow:Enabling eager execution
INFO:tensorflow:Enabling v2 tensorshape
INFO:tensorflow:Enabling resource variables
INFO:tensorflow:Enabling tensor equality
INFO:tensorflow:Enabling control flow v2


In [2]:
#importTrainData()
train_data = np.load("../train_40000.npy")
params_data = np.load("../train_target_40000.npy")
train_parameter = np.load("../train_parameter_40000.npy")
#
validation_data = np.load("../val_train_40000.npy")
validation_params = np.load("../val_target_40000.npy")
validation_parameter = np.load("../val_parameter_40000.npy")

In [3]:
mean_value_target = np.mean(params_data)
standard_deviation_target = np.std(params_data)

mean_value = np.mean(train_data)
standard_deviation = np.std(train_data)
measurements_to_remove = 30

In [4]:
median_train = False
replaceValuesBiggerOne_train=False
replaceFirstXValues_train=False
removeFirstXValues_train=True
normalizeData_train=False
zScore_train=True

In [5]:
train_data = preprocessData(train_data,
                          median=median_train,
                          replaceValuesBiggerOne = replaceValuesBiggerOne_train,
                          replaceFirstXValues = replaceFirstXValues_train,
                          removeFirstXValues = removeFirstXValues_train,
                          normalizeData = normalizeData_train,
                          zScore = zScore_train,
                          zScore_mean = mean_value,
                          zScore_sd = standard_deviation,
                          number_to_remove = measurements_to_remove)

train_target = preprocessData(params_data,
                          median=False,
                          replaceValuesBiggerOne=False,
                          replaceFirstXValues=False,
                          removeFirstXValues=False,
                          normalizeData=False,
                          zScore=False,
                          zScore_mean=mean_value_target,
                          zScore_sd=standard_deviation_target,
                          number_to_remove=measurements_to_remove)

validation_data = preprocessData(validation_data,
                          median = median_train,
                          replaceValuesBiggerOne = replaceValuesBiggerOne_train,
                          replaceFirstXValues = replaceFirstXValues_train,
                          removeFirstXValues = removeFirstXValues_train,
                          normalizeData = normalizeData_train,
                          zScore = zScore_train,
                          zScore_mean = mean_value,
                          zScore_sd = standard_deviation,
                          number_to_remove = measurements_to_remove)

validation_target = preprocessData(validation_params,
                          median=False,
                          replaceValuesBiggerOne=False,
                          replaceFirstXValues=False,
                          removeFirstXValues=False,
                          normalizeData=False,
                          zScore=False,
                          zScore_mean=mean_value_target,
                          zScore_sd=standard_deviation_target,
                          number_to_remove=measurements_to_remove)

In [6]:
def scaleToMaxToOne(dat, min, max):
    scaled_dat = dat / max
    return scaled_dat

def unscaleMaxToOne(scaled_dat, min, max):
    unscaled_dat = scaled_dat * max
    return unscaled_dat

min_target = np.min(train_target) 
max_target = np.max(train_target)
print(f"Rescaling targets with max = {max_target}")
train_target = scaleToMaxToOne(train_target, min_target, max_target)
val_target = scaleToMaxToOne(validation_target, min_target, max_target)


Rescaling targets with max = 0.6713607926990712


In [7]:
number_of_measurements = 300
if removeFirstXValues_train == True:
    number_of_measurements = 300 - measurements_to_remove    
    
#reshaping train data
if type(train_data) != np.ndarray:
    train_data = train_data.to_numpy().reshape(-1, 55, number_of_measurements)
else:
    train_data = train_data.reshape(-1, 55, number_of_measurements)

#reshaping target data
if type(params_data) != np.ndarray:
    params_data = params_data.to_numpy()

#reshaping val data
if type(validation_data) != np.ndarray:
    validation_data = validation_data.to_numpy().reshape(-1, 55, number_of_measurements)
else:    
    validation_data = validation_data.reshape(-1, 55, number_of_measurements)

#reshaping val target
if type(validation_params) != np.ndarray:
    validation_params = validation_params.to_numpy()


In [8]:
prevent_overfitting = keras.callbacks.EarlyStopping(monitor="val_loss", patience = 20, restore_best_weights=True)
reduce_lr = keras.callbacks.ReduceLROnPlateau(monitor='val_loss', patience=10, min_delta=0.0001)

In [9]:
def logistic_regression(av = 'selu'):
    first_input = Input(shape=(55, number_of_measurements))
    flatten_layer = tf.keras.layers.Flatten()(first_input)
    output = Dense(55)(flatten_layer)


    model = Model(inputs=first_input, outputs=output)
    optim = keras.optimizers.Adam(learning_rate=0.0001)
    
    model.compile(optimizer=optim,
                  loss='Huber')
        
    return model

model = logistic_regression()
model2 = logistic_regression()
model3 = logistic_regression()

# Model 1

In [10]:
history_test = model.fit(train_data,
                    train_target,
                    batch_size=32,
                    epochs = 1000,
                    #validation_data= (validation_data, validation_params),
                    validation_split = 0.2,
                    shuffle = True,
                    callbacks=[prevent_overfitting, reduce_lr])

Epoch 1/1000
1000/1000 [==============================] - 4s 3ms/step - loss: 0.0481 - val_loss: 0.0350
Epoch 2/1000
1000/1000 [==============================] - 2s 2ms/step - loss: 0.0173 - val_loss: 0.0254
Epoch 3/1000
1000/1000 [==============================] - 2s 2ms/step - loss: 0.0085 - val_loss: 0.0215
Epoch 4/1000
1000/1000 [==============================] - 2s 2ms/step - loss: 0.0051 - val_loss: 0.0174
Epoch 5/1000
1000/1000 [==============================] - 2s 2ms/step - loss: 0.0034 - val_loss: 0.0147
Epoch 6/1000
1000/1000 [==============================] - 2s 2ms/step - loss: 0.0024 - val_loss: 0.0135
Epoch 7/1000
1000/1000 [==============================] - 2s 2ms/step - loss: 0.0019 - val_loss: 0.0133
Epoch 8/1000
1000/1000 [==============================] - 2s 2ms/step - loss: 0.0016 - val_loss: 0.0130
Epoch 9/1000
1000/1000 [==============================] - 2s 2ms/step - loss: 0.0015 - val_loss: 0.0116
Epoch 10/1000
1000/1000 [==============================] - 2s 2m

In [11]:
pred = model.predict(validation_data)
#pred2 = pred2 * standard_deviation_target + mean_value_target 
#score2 = scoringSystem(validation_params, pred2)

pred_final = unscaleMaxToOne(pred, min_target, max_target)
score = scoringSystem(validation_params, pred_final)
print(score)

tf.Tensor(550000.0, shape=(), dtype=float64)
tf.Tensor(6792.37294519486, shape=(), dtype=float64)


In [12]:
last_five_idx = len(history_test.history["val_loss"])-5
val_loss_1 = sum(history_test.history["val_loss"][last_five_idx:])/5

In [26]:
last_five_idx = len(history_test.history["loss"])-5
train_loss_1 = sum(history_test.history["loss"][last_five_idx:])/5

# Model 2

In [14]:
history_test_2 = model2.fit(train_data,
                    train_target,
                    batch_size=32,
                    epochs = 1000,
                    #validation_data= (validation_data, validation_params),
                    validation_split = 0.2,
                    shuffle = True,
                    callbacks=[prevent_overfitting, reduce_lr])

Epoch 1/1000
1000/1000 [==============================] - 3s 3ms/step - loss: 0.0474 - val_loss: 0.0344
Epoch 2/1000
1000/1000 [==============================] - 2s 2ms/step - loss: 0.0169 - val_loss: 0.0259
Epoch 3/1000
1000/1000 [==============================] - 2s 2ms/step - loss: 0.0085 - val_loss: 0.0211
Epoch 4/1000
1000/1000 [==============================] - 2s 2ms/step - loss: 0.0051 - val_loss: 0.0181
Epoch 5/1000
1000/1000 [==============================] - 2s 2ms/step - loss: 0.0033 - val_loss: 0.0162
Epoch 6/1000
1000/1000 [==============================] - 2s 2ms/step - loss: 0.0024 - val_loss: 0.0145
Epoch 7/1000
1000/1000 [==============================] - 2s 2ms/step - loss: 0.0019 - val_loss: 0.0126
Epoch 8/1000
1000/1000 [==============================] - 2s 2ms/step - loss: 0.0016 - val_loss: 0.0126
Epoch 9/1000
1000/1000 [==============================] - 2s 2ms/step - loss: 0.0015 - val_loss: 0.0138
Epoch 10/1000
1000/1000 [==============================] - 2s 2m

In [15]:
pred2 = model2.predict(validation_data)
#pred2 = pred2 * standard_deviation_target + mean_value_target 
#score2 = scoringSystem(validation_params, pred2)

pred_final2 = unscaleMaxToOne(pred2, min_target, max_target)
score2 = scoringSystem(validation_params, pred_final2)
print(score2)

tf.Tensor(550000.0, shape=(), dtype=float64)
tf.Tensor(7203.56721965812, shape=(), dtype=float64)


In [27]:
last_five_idx_2 = len(history_test_2.history["val_loss"])-5
val_loss_2 = sum(history_test_2.history["val_loss"][last_five_idx_2:])/5

last_five_idx = len(history_test_2.history["loss"])-5
train_loss_2 = sum(history_test_2.history["loss"][last_five_idx:])/5

# Model 3

In [17]:
history_test_3 = model3.fit(train_data,
                    train_target,
                    batch_size=32,
                    epochs = 1000,
                    #validation_data= (validation_data, validation_params),
                    validation_split = 0.2,
                    shuffle = True,
                    callbacks=[prevent_overfitting, reduce_lr])

Epoch 1/1000
1000/1000 [==============================] - 2s 2ms/step - loss: 0.0479 - val_loss: 0.0352
Epoch 2/1000
1000/1000 [==============================] - 2s 2ms/step - loss: 0.0170 - val_loss: 0.0255
Epoch 3/1000
1000/1000 [==============================] - 2s 2ms/step - loss: 0.0086 - val_loss: 0.0212
Epoch 4/1000
1000/1000 [==============================] - 2s 2ms/step - loss: 0.0050 - val_loss: 0.0185
Epoch 5/1000
1000/1000 [==============================] - 2s 2ms/step - loss: 0.0033 - val_loss: 0.0166
Epoch 6/1000
1000/1000 [==============================] - 2s 2ms/step - loss: 0.0024 - val_loss: 0.0141
Epoch 7/1000
1000/1000 [==============================] - 2s 2ms/step - loss: 0.0019 - val_loss: 0.0141
Epoch 8/1000
1000/1000 [==============================] - 2s 2ms/step - loss: 0.0016 - val_loss: 0.0131
Epoch 9/1000
1000/1000 [==============================] - 2s 2ms/step - loss: 0.0015 - val_loss: 0.0134
Epoch 10/1000
1000/1000 [==============================] - 2s 2m

In [18]:
pred3 = model3.predict(validation_data)
#pred2 = pred2 * standard_deviation_target + mean_value_target 
#score2 = scoringSystem(validation_params, pred2)

pred_final3 = unscaleMaxToOne(pred3, min_target, max_target)
score3 = scoringSystem(validation_params, pred_final3)
print(score3)

tf.Tensor(550000.0, shape=(), dtype=float64)
tf.Tensor(6893.352905922416, shape=(), dtype=float64)


In [28]:
last_five_idx_3 = len(history_test_3.history["val_loss"])-5
val_loss_3 = sum(history_test_3.history["val_loss"][last_five_idx_3:])/5

last_five_idx = len(history_test_3.history["loss"])-5
train_loss_3 = sum(history_test_3.history["loss"][last_five_idx:])/5

# Evaluation

In [30]:
val_loss_final = (val_loss_1 + val_loss_2 + val_loss_3)/3
train_loss_final = (train_loss_1 + train_loss_2 + train_loss_3)/3
score_final = (score + score2 + score3)/3

In [31]:
print("Final validation loss: ", val_loss_final)
print("Final train loss: ", train_loss_final)
print("Final Score: ", score_final)

Final validation loss:  0.010679955904682478
Final train loss:  0.00011614027947265034
Final Score:  tf.Tensor(6963.097690258465, shape=(), dtype=float64)


# test

In [22]:
def logistic_regression_dropout(av = 'selu'):
    first_input = Input(shape=(55, number_of_measurements))
    flatten_layer = tf.keras.layers.Flatten()(first_input)
    dropout_layer = tf.keras.layers.Dropout(0.2)(flatten_layer)

    output = Dense(55)(dropout_layer)


    model = Model(inputs=first_input, outputs=output)
    optim = keras.optimizers.Adam(learning_rate=0.0001)
    
    model.compile(optimizer=optim,
                  loss='Huber')
        
    return model

model_test = logistic_regression_dropout()

In [23]:
history_test_test = model_test.fit(train_data,
                    train_target,
                    batch_size=32,
                    epochs = 1000,
                    #validation_data= (validation_data, validation_params),
                    validation_split = 0.2,
                    shuffle = True,
                    callbacks=[prevent_overfitting, reduce_lr])

Epoch 1/1000
1000/1000 [==============================] - 3s 2ms/step - loss: 0.0633 - val_loss: 0.0344
Epoch 2/1000
1000/1000 [==============================] - 2s 2ms/step - loss: 0.0287 - val_loss: 0.0224
Epoch 3/1000
1000/1000 [==============================] - 2s 2ms/step - loss: 0.0157 - val_loss: 0.0174
Epoch 4/1000
1000/1000 [==============================] - 2s 2ms/step - loss: 0.0097 - val_loss: 0.0137
Epoch 5/1000
1000/1000 [==============================] - 2s 2ms/step - loss: 0.0062 - val_loss: 0.0113
Epoch 6/1000
1000/1000 [==============================] - 2s 2ms/step - loss: 0.0041 - val_loss: 0.0095
Epoch 7/1000
1000/1000 [==============================] - 2s 2ms/step - loss: 0.0029 - val_loss: 0.0092
Epoch 8/1000
1000/1000 [==============================] - 2s 2ms/step - loss: 0.0021 - val_loss: 0.0090
Epoch 9/1000
1000/1000 [==============================] - 2s 2ms/step - loss: 0.0016 - val_loss: 0.0061ETA: 1s  - ETA: 0s - loss - ETA: 0s - loss: 0.0
Epoch 10/1000
100

In [24]:
pred_test = model_test.predict(validation_data)
#score_test = scoringSystem(validation_params, pred_test)

pred_final_test = unscaleMaxToOne(pred_test, min_target, max_target)
score_test = scoringSystem(validation_params, pred_final_test)
print(score_test)

tf.Tensor(550000.0, shape=(), dtype=float64)
tf.Tensor(8114.161242161421, shape=(), dtype=float64)


In [25]:
last_five_idx = len(history_test_test.history["val_loss"])-5
sum(history_test_test.history["val_loss"][last_five_idx:])/5

0.006956708896905184